In [1]:
# # For COLAB
# from google.colab import drive
# drive.mount('/content/gdrive')

In [1]:
# !pip install -Uq kaggle torchsummary matplotlib tensorboard scikit-learn
# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download xhlulu/140k-real-and-fake-faces

# import shutil
# shutil.unpack_archive('140k-real-and-fake-faces.zip')

Dataset URL: https://www.kaggle.com/datasets/xhlulu/140k-real-and-fake-faces
License(s): other


In [2]:
import os
import numpy as np
import datetime
from copy import deepcopy
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision.utils import save_image
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchsummary import summary
torch.manual_seed(69)
%load_ext tensorboard

In [4]:
# Hyperparameters
CHECKPOINT_GEN = "wgan_generator_128_2.pth"
CHECKPOINT_CRITIC = "wgan_critic_128_2.pth"
LOG_FOLDER = "logs/"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_EPOCHS = 10
BATCH_SIZE = 512
TRAIN_SUBSET_SIZE = 25000
VALID_SUBSET_SIZE = 5000
TEST_SUBSET_SIZE = 5000
LEARNING_RATE = 1e-4
IMAGE_SIZE = 128
CHANNELS_IMG = 3
Z_DIM = 256
FEATURES_CRITIC = 16
FEATURES_GEN = 16
NUM_WORKERS = 2
# os.makedirs("/content/gdrive/MyDrive/generated-image-detection/models/", exist_ok=True)
os.makedirs(LOG_FOLDER, exist_ok=True)

# Data

In [5]:
def create_dataloader(
    data_path,
    subset_size,
    batch_size,
    img_size,
    only_real=True
):
    data_transforms = transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.Normalize(
           [0.5 for _ in range(3)],
           [0.5 for _ in range(3)]),
    ])

    dataset = datasets.ImageFolder(root=data_path, transform=data_transforms)
    if only_real:
      class_idx = dataset.class_to_idx["real"]
      class_indices = [i for i, label in enumerate(dataset.targets) if label == class_idx]
      subset_indices = class_indices[:subset_size]
    else:
      real_class_idx = dataset.class_to_idx["real"]
      real_class_indices = [
          i for i, label in enumerate(dataset.targets) if label == real_class_idx
      ]
      real_subset_indices = real_class_indices[:subset_size//2]

      fake_class_idx = dataset.class_to_idx["fake"]
      fake_class_indices = [
          i for i, label in enumerate(dataset.targets) if label == fake_class_idx
      ]
      fake_subset_indices = fake_class_indices[:subset_size//2]
      subset_indices = real_subset_indices + fake_subset_indices

    subset_dataset = Subset(dataset, subset_indices)
    dataloader = DataLoader(
       subset_dataset,
       batch_size=batch_size,
       shuffle=True,
       num_workers=NUM_WORKERS,
       pin_memory=True,
       drop_last=True
    )
    return dataloader

In [6]:
train_loader = create_dataloader("real_vs_fake/real-vs-fake/train", TRAIN_SUBSET_SIZE, BATCH_SIZE, IMAGE_SIZE)
valid_loader = create_dataloader("real_vs_fake/real-vs-fake/valid", VALID_SUBSET_SIZE, BATCH_SIZE , IMAGE_SIZE)
test_loader = create_dataloader("real_vs_fake/real-vs-fake/test", TEST_SUBSET_SIZE, BATCH_SIZE , IMAGE_SIZE)

print(f"Training \t Batches: {len(train_loader)} \t Images: {len(train_loader.dataset)}")
print(f"Validation \t Batches: {len(valid_loader)} \t Images: {len(valid_loader.dataset)}")
print(f"Test \t\t Batches: {len(test_loader)} \t Images: {len(test_loader.dataset)}")

Training 	 Batches: 48 	 Images: 25000
Validation 	 Batches: 9 	 Images: 5000
Test 		 Batches: 9 	 Images: 5000


In [7]:
# For generators
fixed_train_vector = torch.randn(len(train_loader), train_loader.batch_size, Z_DIM, 1, 1).to(DEVICE)
fixed_valid_vector = torch.randn(len(valid_loader), valid_loader.batch_size, Z_DIM, 1, 1).to(DEVICE)
fixed_test_vector = torch.randn(len(test_loader), test_loader.batch_size, Z_DIM, 1, 1).to(DEVICE)

# Utils

In [8]:
def save_checkpoint(model, optimizer, epoch=None, loss=None, filename="my_checkpoint.pth.tar"):
    torch.save({
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch,
        'loss': loss,
    }, filename)
    print("=> Model saved")


def load_checkpoint(checkpoint_file, model, optimizer=None, lr=None):
    checkpoint = torch.load(checkpoint_file, map_location="cuda", weights_only=True)
    model.load_state_dict(checkpoint["state_dict"])

    if optimizer is not None:
        optimizer.load_state_dict(checkpoint["optimizer"])

        # If we don't do this then it will just have learning rate of old checkpoint
        # and it will lead to many hours of debugging \:
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr
    print("=> Loaded checkpoint")

## Train Function

In [11]:
import torch
import torch.utils.data
import torch.utils.tensorboard
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score
)
from typing import Optional, Tuple, List

def calculate_metrics(targets: List, predictions: List, scores: List) -> dict:
    """Calculate and return all evaluation metrics."""
    return {
        'accuracy': accuracy_score(targets, predictions),
        'precision': precision_score(targets, predictions, zero_division=1),
        'recall': recall_score(targets, predictions, zero_division=1),
        'f1': f1_score(targets, predictions, zero_division=1),
        'roc_auc': roc_auc_score(targets, scores)
    }

def process_batch(
    disc: torch.nn.Module,
    real_data: torch.Tensor,
    real_labels: torch.Tensor,
    gen: torch.nn.Module,
    fixed_vector: torch.Tensor,
    criterion: torch.nn.Module,
    device: torch.device
) -> Tuple[torch.Tensor, list, list, list]:
    """Process a single batch of data."""
    torch.cuda.empty_cache()
    # Process real samples
    real_outputs = disc(real_data).squeeze()
    real_loss = criterion(real_outputs, real_labels)

    # Generate and process fake samples
    fake = gen(fixed_vector)
    fake_labels = torch.zeros_like(real_labels).to(device).float()
    fake_outputs = disc(fake).squeeze()
    fake_loss = criterion(fake_outputs, fake_labels)

    # print(real_labels[:5], fake_labels[:5])
    # Combine predictions and targets
    scores = torch.cat((real_outputs, fake_outputs)).cpu().detach().numpy()
    targets = torch.cat((real_labels, fake_labels)).cpu().detach().numpy()
    preds = (scores >= 0.5).astype(int) # Converting to discreet 1s and 0s instead of probability
    # print(preds[:5], scores[:5], targets[:5])
    return real_loss + fake_loss, preds, scores, targets

def train_epoch(
    disc: torch.nn.Module,
    gen: torch.nn.Module,
    data_loader: torch.utils.data.DataLoader,
    optimizer: Optional[torch.optim.Optimizer],
    criterion: torch.nn.Module,
    fixed_vector: torch.Tensor,
    device: torch.device,
    phase: str
) -> dict:
    """Run one epoch of training or validation."""
    running_loss = 0.0
    all_preds, all_scores, all_targets = [], [], []

    loop = tqdm(data_loader, leave=True, desc=phase)
    for batch_idx, (real, real_labels) in enumerate(loop):
        real = real.to(device)
        real_labels = real_labels.to(device).float()

        loss, preds, scores, targets = process_batch(
            disc, real, real_labels, gen,
            fixed_vector[batch_idx], criterion, device
        )


        if optimizer is not None:  # Training phase
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Update metrics
        running_loss += loss.item()
        all_preds.extend(preds)
        all_scores.extend(scores)
        all_targets.extend(targets)

        # Update progress bar
        metrics = calculate_metrics(all_targets, all_preds, all_scores)
        metrics['loss'] = running_loss / (batch_idx + 1)
        loop.set_postfix(metrics)

    # Calculate final metrics
    metrics = calculate_metrics(all_targets, all_preds, all_scores)
    metrics['loss'] = running_loss / len(data_loader)
    return metrics

def log_metrics(
    writer: torch.utils.tensorboard.SummaryWriter,
    train_metrics: dict,
    valid_metrics: dict,
    epoch: int
) -> None:
    """Log metrics to TensorBoard."""
    for metric_name in train_metrics:
        writer.add_scalar(f"{metric_name}/train", train_metrics[metric_name], global_step=epoch)
        writer.add_scalar(f"{metric_name}/valid", valid_metrics[metric_name], global_step=epoch)

def train_discriminator(
    disc: torch.nn.Module,
    gen: torch.nn.Module,
    writer: torch.utils.tensorboard.SummaryWriter,
    train_loader: torch.utils.data.DataLoader,
    valid_loader: torch.utils.data.DataLoader,
    fixed_train_vector: torch.Tensor,
    fixed_valid_vector: torch.Tensor,
    num_epochs: int = 100,
    device: torch.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    criterion: torch.nn.Module = torch.nn.BCELoss(),
    optimizer: Optional[torch.optim.Optimizer] = None,
    checkpoint_file: str = "discriminator_checkpoint.pth"
) -> Tuple[torch.nn.Module, List[float], List[float]]:
    """
    Train a discriminator model in a GAN setup.

    Args:
        disc: Discriminator model
        gen: Generator model
        writer: TensorBoard writer
        train_loader: Training data loader
        valid_loader: Validation data loader
        fixed_train_vector: Fixed noise vector for generating fake training samples
        fixed_valid_vector: Fixed noise vector for generating fake validation samples
        num_epochs: Number of training epochs
        device: Device to train on
        criterion: Loss function
        optimizer: Optimizer (defaults to Adam if None)
        checkpoint_file: Path to save model checkpoints

    Returns:
        Tuple of (trained discriminator, training losses, validation losses)
    """
    optimizer = optimizer or optim.Adam(disc.parameters())
    train_losses, valid_losses = [], []
    least_loss = float('inf')
    gen.eval()

    for epoch in range(num_epochs):
        print(f"Epoch [{epoch + 1}/{num_epochs}]")

        # Training phase
        disc.train()
        train_metrics = train_epoch(
            disc, gen, train_loader, optimizer, criterion,
            fixed_train_vector, device, "Train"
        )

        # Validation phase
        disc.eval()
        with torch.no_grad():
            valid_metrics = train_epoch(
                disc, gen, valid_loader, None, criterion,
                fixed_valid_vector, device, "Validation"
            )

        # Log metrics and save checkpoint
        log_metrics(writer, train_metrics, valid_metrics, epoch)
        train_losses.append(train_metrics['loss'])
        valid_losses.append(valid_metrics['loss'])

        if valid_metrics['loss'] < least_loss:
          least_loss = valid_metrics['loss']
          save_checkpoint(disc, optimizer, epoch, least_loss, checkpoint_file)

    return disc, train_losses, valid_losses

## Test Function

In [12]:
def evaluate_model(discriminator, generator, test_loader, fixed_test_vector, device='cuda'):
    """
    Evaluates a discriminator on both real and generated samples.

    Args:
        discriminator: Discriminator model in eval mode
        generator: Generator model
        test_loader: DataLoader for real samples
        fixed_test_vector: Latent vectors for generating fake samples
        device: Device to run evaluation on

    Returns:
        dict: Evaluation metrics
    """
    discriminator.eval()
    generator.eval()
    criterion = torch.nn.BCELoss()
    running_loss = 0.0
    preds, scores, targets = [], [], []

    with torch.no_grad():
        loop = tqdm(test_loader, leave=True, desc="Testing")
        for batch_idx, (real, real_labels) in enumerate(loop):
            real = real.to(device)
            real_labels = real_labels.to(device).float()
            real_outputs = discriminator(real).squeeze()
            real_loss = criterion(real_outputs, real_labels)

            fake = generator(fixed_test_vector[batch_idx])
            fake_labels = torch.zeros_like(real_labels).to(device).float()
            fake_outputs = discriminator(fake).squeeze()
            fake_loss = criterion(fake_outputs, fake_labels)

            loss = real_loss + fake_loss
            running_loss += loss.item()

            outputs = torch.cat((real_outputs, fake_outputs), dim=0).cpu().numpy()
            labels = torch.cat((real_labels, fake_labels), dim=0).cpu().numpy()
            scores = np.concatenate((scores, outputs))
            targets = np.concatenate((targets, labels))
            preds = (scores >= 0.5).astype(int) # Converting to discreet 1s and 0s instead of probability
            metrics = {
                'loss': running_loss / (batch_idx + 1),
                'accuracy': accuracy_score(targets, preds),
                'precision': precision_score(targets, preds, zero_division=1),
                'recall': recall_score(targets, preds, zero_division=1),
                'f1': f1_score(targets, preds, zero_division=1),
                'roc_auc': roc_auc_score(targets, scores)
            }

            loop.set_postfix(**metrics)

    final_metrics = {
        k: v/len(test_loader) if k == 'loss' else v
        for k, v in metrics.items()
    }

    print(f"Test: Loss: {final_metrics['loss']:.4f} | "
          f"Accuracy: {final_metrics['accuracy']:.4f} | "
          f"Precision: {final_metrics['precision']:.4f} | "
          f"Recall: {final_metrics['recall']:.4f} | "
          f"F1: {final_metrics['f1']:.4f} | "
          f"ROC AUC: {final_metrics['roc_auc']:.4f}")

    return final_metrics

## Abalation function

In [13]:
import copy

def abalation(discriminator, generator, no_target_layers):
  time = datetime.datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
  log_folder = os.path.join(LOG_FOLDER, f"wgan_abalation/{time}")
  writer = SummaryWriter(log_folder, comment=f"WGAN discriminator {time}")

  for param in discriminator.parameters():
      param.requires_grad = False

  layers = None

  results = {}

  for n in range(1, len(no_target_layers) + 1):
    CHECKPOINT_WGAN_DISC = f"models/wgan_{n}unfreeze.pth"
    for param in layers[-n].parameters():
      param.requires_grad = True

    discriminator_copy = copy.deepcopy(discriminator)

    discriminator_copy, wgan_disc_train_losses, wgan_disc_valid_losses = train_discriminator(
      disc=discriminator_copy,
      gen=generator,
      writer=writer,
      train_loader=train_loader,
      valid_loader=valid_loader,
      fixed_train_vector=fixed_train_vector,
      fixed_valid_vector=fixed_valid_vector,
      num_epochs=NUM_EPOCHS,
      device=DEVICE,
      criterion=torch.nn.BCELoss(),
      optimizer=optim.Adam(wgan_disc.parameters(), lr=LEARNING_RATE, betas=(0.0, 0.9)),
      checkpoint_file=CHECKPOINT_WGAN_DISC,
    )

    load_checkpoint(CHECKPOINT_WGAN_DISC, discriminator_copy)

    wgan_disc_test_metrics = evaluate_model(
      discriminator=discriminator_copy,
      generator=generator,
      test_loader=test_loader,
      fixed_test_vector=fixed_test_vector,
    )

    results[CHECKPOINT_WGAN_DISC] = wgan_disc_test_metrics
  return results

# Models

In [14]:
class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            # Input: N x channels_noise x 1 x 1
            self._block(channels_noise, features_g * 32, 4, 1, 0),  # img: 4
            self._block(features_g * 32, features_g * 16, 4, 2, 1),  # img: 8
            self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 16
            self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 32
            self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 64
            self._block(features_g * 2, features_g * 1, 4, 2, 1),  # img: 128
            nn.Conv2d(
                features_g * 1, channels_img, kernel_size=3, stride=1, padding=1
            ),
            # Output: N x channels_img x 128 x 128
            nn.Tanh(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)

wgan_gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN).to(DEVICE)
load_checkpoint(CHECKPOINT_GEN, wgan_gen)
wgan_gen.eval()

=> Loaded checkpoint


Generator(
  (net): Sequential(
    (0): Sequential(
      (0): ConvTranspose2d(256, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): Sequential(
      (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): Sequential(
      (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (4): Sequential(
      (0): ConvTr

In [15]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # input: N x channels_img x 128 x 128
            nn.Conv2d(channels_img, features_d, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2),
            # _block(in_channels, out_channels, kernel_size, stride, padding)
            self._block(features_d, features_d * 2, 4, 2, 1),
            self._block(features_d * 2, features_d * 4, 4, 2, 1),
            self._block(features_d * 4, features_d * 8, 4, 2, 1),
            self._block(features_d * 8, features_d * 16, 4, 2, 1),
            self._block(features_d * 16, features_d * 32, 4, 2, 1),
            # After all _block img output is 4x4 (Conv2d below makes into 1x1)
            nn.Conv2d(features_d * 32, features_d * 32, kernel_size=4, stride=1, padding=0, bias=False),
            nn.LeakyReLU(0.2),
            nn.Conv2d(features_d * 32, 1, kernel_size=1, stride=1, padding=0, bias=False),
            nn.Flatten(),
            nn.Sigmoid(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.InstanceNorm2d(out_channels, affine=True),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)

wgan_disc = Discriminator(CHANNELS_IMG, FEATURES_CRITIC).to(DEVICE)
load_checkpoint(CHECKPOINT_CRITIC, wgan_disc)
for param in wgan_disc.parameters():
    param.requires_grad = True
wgan_disc.train()

=> Loaded checkpoint


Discriminator(
  (disc): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (5): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=

In [16]:
import os
import copy
import datetime
import torch
from torch.utils.tensorboard import SummaryWriter
from typing import Dict, List
from tqdm import tqdm

# Assumes the following are defined externally:
#  discriminator, generator: pretrained models
#  train_loader, valid_loader, test_loader: DataLoader
#  fixed_train_vector, fixed_valid_vector, fixed_test_vector: torch.Tensor noise vectors
#  DEVICE: torch.device
#  NUM_EPOCHS: int
#  LEARNING_RATE: float
#  LOG_FOLDER: str
#  train_discriminator, evaluate_model are imported

def ablation_study_incremental(discriminator: torch.nn.Module,
                               generator: torch.nn.Module,
                               train_loader,
                               valid_loader,
                               test_loader,
                               fixed_train_vector: torch.Tensor,
                               fixed_valid_vector: torch.Tensor,
                               fixed_test_vector: torch.Tensor,
                               device: torch.device,
                               num_epochs: int,
                               learning_rate: float,
                               log_folder: str) -> Dict[str, dict]:
    """
    Conduct incremental ablation where:
      1) First, unfreeze only the penultimate of the last 5 layers
      2) Then unfreeze the last 2 layers
      3) Then last 3, up to all 5
    Train only those layers each run and evaluate on the test set.

    Returns a dict mapping descriptive labels to test metrics.
    """
    # Extract all parameterized modules from the Sequential
    all_modules = list(discriminator.disc.children())
    param_modules = [m for m in all_modules if sum(p.numel() for p in m.parameters()) > 0]
    last_five = param_modules[-5:]

    results: Dict[str, dict] = {}
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    # Iterate over configurations with progress bar
    for n in tqdm(range(1, len(last_five) + 1), desc="Ablation configs", unit="config"):
        # Determine which modules to unfreeze
        if n == 1:
            modules_to_unfreeze = last_five[-2:-1]
            label = "penultimate_only"
        else:
            modules_to_unfreeze = last_five[-n:]
            label = f"last_{n}_layers"

        # Deep copy and freeze all parameters
        disc_copy = copy.deepcopy(discriminator).to(device)
        for p in disc_copy.parameters():
            p.requires_grad = False

        # Map copies of parameterized modules
        modules_copy = list(disc_copy.disc.children())
        param_copy = [m for m in modules_copy if sum(p.numel() for p in m.parameters()) > 0]

        # Slice target copies
        if n == 1:
            target_copies = param_copy[-2:-1]
        else:
            target_copies = param_copy[-n:]

        # Unfreeze target modules
        for module in target_copies:
            for p in module.parameters():
                p.requires_grad = True

        # Collect trainable params
        trainable = [p for p in disc_copy.parameters() if p.requires_grad]
        if not trainable:
            raise ValueError(f"No parameters to optimize for config '{label}'")

        # Optimizer for unfreezed params
        optimizer = torch.optim.Adam(trainable, lr=learning_rate, betas=(0.0, 0.9))

        # TensorBoard logging
        log_dir = os.path.join(log_folder, f"ablation_{label}_{timestamp}")
        writer = SummaryWriter(log_dir)

        # Train
        disc_copy, train_losses, valid_losses = train_discriminator(
            disc=disc_copy,
            gen=generator,
            writer=writer,
            train_loader=train_loader,
            valid_loader=valid_loader,
            fixed_train_vector=fixed_train_vector,
            fixed_valid_vector=fixed_valid_vector,
            num_epochs=num_epochs,
            device=device,
            criterion=torch.nn.BCELoss(),
            optimizer=optimizer,
            checkpoint_file=os.path.join(log_folder, f"disc_ablation_{label}.pth")
        )

        # Test
        test_metrics = evaluate_model(
            discriminator=disc_copy,
            generator=generator,
            test_loader=test_loader,
            fixed_test_vector=fixed_test_vector,
            device=device
        )

        results[label] = test_metrics
        writer.close()

    return results

# Example usage:
results = ablation_study_incremental(
    discriminator=wgan_disc,
    generator=wgan_gen,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    fixed_train_vector=fixed_train_vector,
    fixed_valid_vector=fixed_valid_vector,
    fixed_test_vector=fixed_test_vector,
    device=DEVICE,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    log_folder=LOG_FOLDER
)
print(results)


Ablation configs:   0%|          | 0/5 [00:00<?, ?config/s]

Epoch [1/10]



Train: 100%|██████████| 48/48 [00:18<00:00,  2.60it/s, accuracy=0.622, precision=0.882, recall=0.283, f1=0.428, roc_auc=0.566, loss=63.8]

Validation: 100%|██████████| 9/9 [00:03<00:00,  2.28it/s, accuracy=0.894, precision=0.916, recall=0.867, f1=0.891, roc_auc=0.96, loss=1.91]


=> Model saved
Epoch [2/10]



Train: 100%|██████████| 48/48 [00:17<00:00,  2.77it/s, accuracy=0.91, precision=0.91, recall=0.911, f1=0.911, roc_auc=0.968, loss=1.38]

Validation: 100%|██████████| 9/9 [00:03<00:00,  2.27it/s, accuracy=0.928, precision=0.967, recall=0.885, f1=0.924, roc_auc=0.982, loss=0.953]


...

=> Model saved
Epoch [9/10]



Train: 100%|██████████| 48/48 [00:17<00:00,  2.78it/s, accuracy=0.942, precision=0.927, recall=0.96, f1=0.943, roc_auc=0.961, loss=5.78]

Validation: 100%|██████████| 9/9 [00:03<00:00,  2.42it/s, accuracy=0.982, precision=0.967, recall=0.997, f1=0.982, roc_auc=0.999, loss=0.289]


Epoch [10/10]



Train: 100%|██████████| 48/48 [00:17<00:00,  2.80it/s, accuracy=0.921, precision=0.898, recall=0.949, f1=0.923, roc_auc=0.944, loss=8.32]

Validation: 100%|██████████| 9/9 [00:03<00:00,  2.37it/s, accuracy=0.988, precision=0.988, recall=0.987, f1=0.988, roc_auc=0.999, loss=0.189]

Ablation configs: 100%|██████████| 5/5 [18:00<00:00, 216.08s/config]

Test: Loss: 0.0324 | Accuracy: 0.9839 | Precision: 0.9844 | Recall: 0.9835 | F1: 0.9839 | ROC AUC: 0.9984
{'penultimate_only': {'loss': 0.028321669793423307, 'accuracy': 0.9811197916666666, 'precision': 0.980702515177797, 'recall': 0.9815538194444444, 'f1': 0.9811279826464209, 'roc_auc': np.float64(0.996413431049865)}, 'last_2_layers': {'loss': 0.02987432038342511, 'accuracy': 0.9791666666666666, 'precision': 0.9825174825174825, 'recall': 0.9756944444444444, 'f1': 0.9790940766550522, 'roc_auc': np.float64(0.9961034751232759)}, 'last_3_layers': {'loss': 0.01948475635346071, 'accuracy': 0.9844835069444444, 'precision': 0.990551527136893, 'recall': 0.9782986111111112, 'f1': 0.984386941805874, 'roc_auc': np.float64(0.9985877378487292)}, 'last_4_layers': {'loss': 11.11111111111111, 'accuracy': 0.5, 'precision': 0.5, 'recall': 1.0, 'f1': 0.6666666666666666, 'roc_auc': np.float64(0.5)}, 'last_5_layers': {'loss': 0.03242345227871412, 'accuracy': 0.9839409722222222, 'precision': 0.9843614248479

In [17]:
results

{'penultimate_only': {'loss': 0.028321669793423307,
  'accuracy': 0.9811197916666666,
  'precision': 0.980702515177797,
  'recall': 0.9815538194444444,
  'f1': 0.9811279826464209,
  'roc_auc': np.float64(0.996413431049865)},
 'last_2_layers': {'loss': 0.02987432038342511,
  'accuracy': 0.9791666666666666,
  'precision': 0.9825174825174825,
  'recall': 0.9756944444444444,
  'f1': 0.9790940766550522,
  'roc_auc': np.float64(0.9961034751232759)},
 'last_3_layers': {'loss': 0.01948475635346071,
  'accuracy': 0.9844835069444444,
  'precision': 0.990551527136893,
  'recall': 0.9782986111111112,
  'f1': 0.984386941805874,
  'roc_auc': np.float64(0.9985877378487292)},
 'last_4_layers': {'loss': 11.11111111111111,
  'accuracy': 0.5,
  'precision': 0.5,
  'recall': 1.0,
  'f1': 0.6666666666666666,
  'roc_auc': np.float64(0.5)},
 'last_5_layers': {'loss': 0.03242345227871412,
  'accuracy': 0.9839409722222222,
  'precision': 0.9843614248479583,
  'recall': 0.9835069444444444,
  'f1': 0.98393399913